In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import csv
import pandas as pd
import json
import time
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import string 
import unicodedata as ud
from greek_stemmer import GreekStemmer
import pymongo
import numpy as np
import re
import networkx as nx

In [ ]:
def create_graph_of_words(words_in_row:list)->nx.DiGraph:
    g = nx.DiGraph()
    #add unique words as nodes of the graph
    g.add_nodes_from(set(words_in_row))
    for j,word in enumerate(words_in_row):
        #add word as a node if it doesnt exist
        #if word not in g.nodes:
            #g.add_node(word)    
        #generate list of indexes
        gen = (x for x in range(j+1,j+4) if x<len(words_in_row))
        for k in gen:
            #avoid self-loops
            if(word == words_in_row[k]):
                pass
            #if edge does not exist create it, else increase edge weight
            elif((word, words_in_row[k]) not in g.edges):
                g.add_edge(word, words_in_row[k])
                g[word][words_in_row[k]]['weight'] = 1
            else:
                g[word][words_in_row[k]]['weight'] += 1
    return g

In [3]:
import pymongo
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
database = mongo_client["GreekParliamentProceedings"]
collection = database["Database"]
#dataframe = pd.DataFrame(list(collection.find()))

In [4]:
#test, pls ignore
dataframe1 = list(collection.find({ }, { "_id": 1, "speech": 1 })[:100000])
for i, row in enumerate(dataframe1):
    #print(row["_id"])
    pass

In [54]:
#test, pls ignore
ticks = [x for x in range(0,1200234,100000)]
ticks.append(1200234)
for i in range(len(ticks)-1):
    #print(ticks[i],ticks[i+1])
    pass
import pymongo
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
inverted_index = mongo_client["GreekParliamentProceedings"]
collection = inverted_index["InvertedIndex"]
for i in collection.find({"list.postinglist":{"$gt":900000}}):
    print(i["_id"])

In [ ]:
#print(graphs[0].edges)
import matplotlib.pyplot as plt
 
plt.savefig('graph.png')
# larger figure size
#pos = nx.circular_layout(graphs[2])

plt.figure(3,figsize=(12,12)) 
nx.draw_networkx(graphs[2], with_labels=True,node_size=80,font_size=15)
plt.show()

In [ ]:
print(len(graphs[456].nodes))
#567

In [ ]:
res = nx.k_core(graphs[456])
print(len(res.nodes))
print((res.nodes))

## Networkx library test

In [ ]:
#networkx library
import networkx as nx
g = nx.Graph()
l = ['1','2','3']
#g.add_nodes_from(l)

for word in l:
    if(word not in list(g.nodes)):
        g.add_node(word)
for e in list(g.nodes):
    print(e)
    
g.add_edge('1','2')
g.add_edge('2','3')
if(('1','2') in g.edges):
    print(' in')
for e in list(g.edges):
    print(e)